# Porting CUDA programs to HIP

HIP API calls are designed to closely match their CUDA equivalents. This enables HIP to function as a thin layer over CUDA and allows for reasonably easy porting of CUDA code to HIP code. Often it is just a matter of replacing **cuda -> hip** in the function calls. The ROCM suite provides two different tools **hipify-perl** and **hipify-clang** to help with the porting process. The tool **hipify-perl** is robust and uses perl to perform an intelligent search and replace of cuda calls with hip calls, while the **hipify-clang** tool uses the clang preprocessor to produce a high quality port. The perl-based method is better for quick ports of small codes, while the clang-based method is intended for ports of large codebases. The hipify-clang tool is much more picky though and fails easily unless it has access to all the header files used in the compilation of the CUDA code.

## Supported API's

A large subset of CUDA API calls are supported by HIP, including those in supporting libraries like **cuBLAS**. Tables in [this Github site](https://github.com/ROCm-Developer-Tools/HIPIFY/blob/amd-staging/docs/supported_apis.md) provides some guidance as to what is supported.

## Setup and installation

From [this source](https://sep5.readthedocs.io/en/latest/Programming_Guides/HIP-porting-guide.html) it is recommended to attempt porting on a machine that has access to both CUDA and HIP libraries. This usually means doing the port on a machine with an NVIDIA GPU. Then one can try porting portions of the code at a time and compare results. For best results with hipify-clang you need to have a version of CUDA that is compatible with your installed version of hipify-clang. 

In [33]:
!hipify-clang --version

AMD LLVM version 16.0.0git
  Optimized build.


Here is a page which describes compatibility between CUDA and hipify-clang.

[HIPIFY Documentation](https://rocm.docs.amd.com/projects/HIPIFY/en/latest/hipify-clang.html)

## Trial setup

There are two sub-directories in this module:

* cuda_mat_mult
* hip_mat_mult

In the directory **cuda_mat_mult** is a CUDA version of the HIP matrix multiplication code in **hip_mat_mult**. It was manually ported from HIP to CUDA. We are going to use the HIP tools to try and port back the CUDA code to HIP code. 

## Porting techniques

### Port a single file

Let's first make a temporary of **cuda_mat_mult** for the purpose of conversion.

In [34]:
!mkdir -p temp_mat_mult; cp -r cuda_mat_mult/* temp_mat_mult/ 

The **hipify-perl** command can port a single file to use the HIP API. We use it to port the file **mat_mult.cu** in the directory **temp_mat_mult**. The flag `-hip-kernel-execution-syntax` changes kernel launch syntax from the CUDA-style triple Chevron `<<< >>>` method to the ANSI C++ compliant method of **hipLaunchKernelGGL**. The following command dumps the output to the command line, but you can use the `-o` flag to specify an output file.

In [38]:
!cd temp_mat_mult; hipify-perl -hip-kernel-execution-syntax mat_mult.cpp

#include "hip/hip_runtime.h"
#include "hip/hip_runtime.h"
/* Code to perform a Matrix multiplication using cuda
Written by Dr Toby M. Potter
*/

// Setup headers
#include <cassert>
#include <cmath>
#include <iostream>

// Bring in the size of the matrices
#include "mat_size.hpp"

// Bring in a library to manage matrices on the CPU
#include "mat_helper.hpp"

// Bring in helper header to manage boilerplate code
#include "cuda_helper.hpp"

// standard matrix multiply kernel 
__global__ void mat_mult (
        float* A, 
        float* B, 
        float* C, 
        size_t N1_A, 
        size_t N0_C,
        size_t N1_C) { 
            
    // A is of size (N0_C, N1_A)
    // B is of size (N1_A, N1_C)
    // C is of size (N0_C, N1_C)   
    
    // i0 and i1 represent the coordinates in Matrix C 
    // We use row-major ordering for the matrices
    
    size_t i0 = blockIdx.y * blockDim.y + threadIdx.y;
    size_t i1 = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Scratch variable
  

If we use the `-inplace` flag, **hipify-perl** copies the file [mat_mult.cpp](temp_mat_mult/mat_mult.cpp) first to [mat_mult.cpp.prehip](temp_mat_mult/mat_mult.cpp.prehip) **if that file doesn't already exist**. Then it performs the conversion from [mat_mult.cpp.prehip](temp_mat_mult/mat_mult.cpp.prehip) to [mat_mult.cpp](temp_mat_mult/mat_mult.cpp). 

In [40]:
!cd temp_mat_mult; hipify-perl -inplace -print-stats -hip-kernel-execution-syntax mat_mult.cpp


[HIPIFY] info: file 'mat_mult.cpp' statistics:
  CONVERTED refs count: 15
  TOTAL lines of code: 190
  WARNINGS: 0
[HIPIFY] info: CONVERTED refs by names:
  cudaDeviceSynchronize => hipDeviceSynchronize: 1
  cudaFree => hipFree: 3
  cudaGetLastError => hipGetLastError: 1
  cudaMalloc => hipMalloc: 3
  cudaMemcpy => hipMemcpy: 3
  cudaMemcpyDeviceToHost => hipMemcpyDeviceToHost: 1
  cudaMemcpyHostToDevice => hipMemcpyHostToDevice: 2


Subsequent edits to [mat_mult.cpp.prehip](temp_mat_mult/mat_mult.cpp.prehip) will be propagated across to [mat_mult.cpp](temp_mat_mult/mat_mult.cpp). This allows for an iterative porting process. Use the `--help` flag for more porting options.

### Examine a directory structure for porting potential

We use the scripts **hipexamine-perl.sh** or **hipexamine.sh** to recursively search through a directory and examine the potential for porting a code. Note the summary that is produced for each file.

In [42]:
!hipexamine-perl.sh cuda_mat_mult -exclude-dirs=".ipynb_checkpoints"


[HIPIFY] info: file 'cuda_mat_mult/mat_mult.cpp' statistics:
  CONVERTED refs count: 14
  TOTAL lines of code: 190
  WARNINGS: 0
[HIPIFY] info: CONVERTED refs by names:
  cudaDeviceSynchronize => hipDeviceSynchronize: 1
  cudaFree => hipFree: 3
  cudaGetLastError => hipGetLastError: 1
  cudaMalloc => hipMalloc: 3
  cudaMemcpy => hipMemcpy: 3
  cudaMemcpyDeviceToHost => hipMemcpyDeviceToHost: 1
  cudaMemcpyHostToDevice => hipMemcpyHostToDevice: 2

[HIPIFY] info: file 'cuda_mat_mult/cuda_helper.hpp' statistics:
  CONVERTED refs count: 55
  TOTAL lines of code: 789
  WARNINGS: 1
[HIPIFY] info: CONVERTED refs by names:
  CUDA_SUCCESS => hipSuccess: 4
  CUresult => hipError_t: 4
  cuGetErrorString => hipDrvGetErrorString: 1
  cuInit => hipInit: 1
  cuda.h => hip/hip_runtime.h: 2
  cudaDevAttrManagedMemory => hipDeviceAttributeManagedMemory: 1
  cudaDeviceGetAttribute => hipDeviceGetAttribute: 1
  cudaDeviceProp => hipDeviceProp_t: 2
  cudaDeviceReset => hipDeviceReset: 1
  cudaDeviceSynchr

If we try the hip-clang version we see that it doesn't handle preprocessor directives very well. The following errors with `_aligned_malloc` are due to it not picking up the windows-specific `#define` clauses.

In [43]:
!hipexamine.sh ./cuda_mat_mult -exclude-dirs=".ipynb_checkpoints"


[HIPIFY] error: hipify-clang: Unknown command line argument '-exclude-dirs=.ipynb_checkpoints'.  Try: '/opt/rocm-5.6.1/bin/hipify-clang --help'
hipify-clang: Did you mean '--o-dir=.ipynb_checkpoints'?



### Porting a directory structure inplace

Both the **hipconvertinplace-perl.sh** and **hipconvertinplace.sh** scripts have the ability to convert a code tree inplace. The additional option **-hip-kernel-execution-syntax** replaces CUDA triple chevron kernel calls with the equivalent call to **hipLaunchKernelGGL** macro.

#### Porting inplace with hipify-perl

In [46]:
!hipconvertinplace-perl.sh temp_mat_mult -hip-kernel-execution-syntax


[HIPIFY] info: file 'temp_mat_mult/mat_mult.cpp' statistics:
  CONVERTED refs count: 15
  TOTAL lines of code: 190
  WARNINGS: 0
[HIPIFY] info: CONVERTED refs by names:
  cudaDeviceSynchronize => hipDeviceSynchronize: 1
  cudaFree => hipFree: 3
  cudaGetLastError => hipGetLastError: 1
  cudaMalloc => hipMalloc: 3
  cudaMemcpy => hipMemcpy: 3
  cudaMemcpyDeviceToHost => hipMemcpyDeviceToHost: 1
  cudaMemcpyHostToDevice => hipMemcpyHostToDevice: 2

[HIPIFY] info: file 'temp_mat_mult/cuda_helper.hpp' statistics:
  CONVERTED refs count: 56
  TOTAL lines of code: 789
  WARNINGS: 1
[HIPIFY] info: CONVERTED refs by names:
  CUDA_SUCCESS => hipSuccess: 4
  CUresult => hipError_t: 4
  cuGetErrorString => hipDrvGetErrorString: 1
  cuInit => hipInit: 1
  cuda.h => hip/hip_runtime.h: 2
  cudaDevAttrManagedMemory => hipDeviceAttributeManagedMemory: 1
  cudaDeviceGetAttribute => hipDeviceGetAttribute: 1
  cudaDeviceProp => hipDeviceProp_t: 2
  cudaDeviceReset => hipDeviceReset: 1
  cudaDeviceSynchr

#### Porting inplace with hipify-clang

Here is the same port with **hipify-clang**.

In [47]:
!hipconvertinplace.sh temp_mat_mult -hip-kernel-execution-syntax

In file included from /tmp/mat_mult.cpp-25db3c.hip:1:
In file included from /opt/rocm-5.6.1/include/hip/hip_runtime.h:64:
In file included from /opt/rocm-5.6.1/include/hip/nvidia_detail/nvidia_hip_runtime.h:28:
In file included from /opt/rocm-5.6.1/include/hip/hip_runtime_api.h:8361:
/opt/rocm-5.6.1/include/hip/nvidia_detail/nvidia_hip_runtime_api.h:2667:34: warning: 'cuCtxDetach' is deprecated [-Wdeprecated-declarations]
    return hipCUResultTohipError(cuCtxDetach(ctx));
                                 ^
/usr/local/cuda-12.1/include/cuda.h:6307:1: note: 'cuCtxDetach' has been explicitly marked deprecated here
__CUDA_DEPRECATED CUresult CUDAAPI cuCtxDetach(CUcontext ctx);
^
/usr/local/cuda-12.1/include/cuda.h:71:42: note: expanded from macro '__CUDA_DEPRECATED'
#define __CUDA_DEPRECATED __attribute__((deprecated))
                                         ^
In file included from /tmp/mat_mult.cpp-25db3c.hip:1:
In file included from /opt/rocm-5.6.1/include/hip/hip_runtime.h:64:
In file

#### Building the ported code

If we examine the source tree we see that every source file that has been hipified has been first copied to a file with suffix `*.prehip`. Then the converted code is overwritten in place of the old file.

In [48]:
!ls -l temp_mat_mult

total 2380
-rw-rw-r-- 1 toby toby  262144 Sep 26 16:29 array_A.dat
-rw-rw-r-- 1 toby toby  262144 Sep 26 16:29 array_B.dat
-rw-rw-r-- 1 toby toby  262144 Sep 26 16:29 array_C.dat
-rw-rw-r-- 1 toby toby   24660 Sep 26 16:35 cuda_helper.hpp
-rw-rw-r-- 1 toby toby   24629 Sep 26 16:34 cuda_helper.hpp.prehip
-rw-rw-r-- 1 toby toby     341 Sep 26 16:29 Makefile
-rw-rw-r-- 1 toby toby    4497 Sep 26 16:35 mat_helper.hpp
-rw-rw-r-- 1 toby toby    4497 Sep 26 16:34 mat_helper.hpp.prehip
-rw-rw-r-- 1 toby toby    5975 Sep 26 16:35 mat_mult.cpp
-rw-rw-r-- 1 toby toby    5944 Sep 26 16:30 mat_mult.cpp.prehip
-rwxrwxr-x 1 toby toby 1545520 Sep 26 16:29 mat_mult.exe
-rw-rw-r-- 1 toby toby     137 Sep 26 16:35 mat_size.hpp
-rw-rw-r-- 1 toby toby     107 Sep 26 16:34 mat_size.hpp.prehip


Try making the ported code with hipcc.

In [51]:
!cd temp_mat_mult; make clean; make CXX="hipcc" LIBFLAGS=""

rm -r *.exe
rm: cannot remove '*.exe': No such file or directory
make: *** [Makefile:20: clean] Error 1
hipcc -g -O2 -x cu mat_mult.cpp -o mat_mult.exe 
cuda_helper.hpp(54): error: function "h_errchk" has already been defined
  void h_errchk(hipError_t errcode, const char* message) {
       ^

1 error detected in the compilation of "mat_mult.cpp".
make: *** [Makefile:16: mat_mult.exe] Error 2


In the original file **cuda_mat_mult/cuda_helper.cpp** we had overloaded the **h_errchk** function to accept errorcodes of both type **CUResult** and **cudaError_t**. Following conversion to HIP the errorcode has been replaced with just **hipError_t**. Therefore we need to manually delete the duplicate **h_errchk** function in **[temp_mat_mult/cuda_helper.hpp.prehip](temp_mat_mult/cuda_helper.hpp.prehip)**. Then rerun the conversion and the make. 

In [54]:
!cd temp_mat_mult; hipify-perl -inplace -hip-kernel-execution-syntax cuda_helper.hpp
!cd temp_mat_mult; make CXX="hipcc" LIBFLAGS=""; ./mat_mult.exe

make: Nothing to be done for 'all'.
Device id: 0
	name:                                    NVIDIA GeForce RTX 3060 Laptop GPU
	global memory size:                      6226 MB
	available registers per block:           65536 
	maximum shared memory size per block:    49 KB
	maximum pitch size for memory copies:    2147 MB
	max block size:                          (1024,1024,64)
	max threads in a block:                  1024
	max Grid size:                           (2147483647,65535,65535)
Maximum error (infinity norm) is: 1.52588e-05


Now we should have a successful port of the CUDA code to HIP!

## Learnings from the porting process

### API differences between CUDA and HIP

CUDA has the notion of a driver API and a runtime API. HIP combines the two into one API and then supports a subset of the combined API. Context managment in HIP is deprecated.

### Tips for managing large kernels

#### Register pressure on ported kernels

Due to compiler and runtime maturity, experience with recent hackathons has shown that the NVIDIA software stack is currently better able to handle kernels with large numbers of registers. When porting to AMD hardware there is likely to be fewer registers available per thread before occupancy is affected. See some of the tips in Lesson 7 on <a href="../L7_Kernel_Optimisation/Optimisation.ipynb">optimising kernels</a> to try and reduce register pressure.

#### Relocatable device code

From [this source](https://docs.amd.com/projects/HIP/en/latest/user_guide/hip_porting_driver_api.html) The linker option `–fgpu-rdc` allows for kernels to call functions that are compiled for different translation units. At the [Pawsey P'Con 23 Hackathon](https://pawsey.org.au/event/pacer-conference-2023-pcon23-registration/) a team found that the use of this flag generated excessively long link times. 